### Extract Data from PDF Files with Python

In [ ]:
# !pip install PDFPlumber
import pdfplumber
import pandas as pd
import os

Ejemplo de Conexion con Pyodbc

In [ ]:
import pyodbc

server = 'ASUSTUF\SQL22'
database = 'AdventureWorksDW2019'
# username = 'nombre_usuario'
# password = 'contraseña'

try:
    connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                                f'SERVER={server};'
                                f'DATABASE={database};'
                                f'Trusted_Connection=yes;')
    print('Conexión exitosa')
except pyodbc.Error as e:
    print(f'Error al conectar a la base de datos: {str(e)}')

Insertar datos mendiate el modulo Cursor

In [ ]:
# Insert data to a SQL Server table using Pyodbc librery with handle errors

# ------ Metodo manual -------
# Create a cursor from the connection
cursor = connection.cursor()
try:
   cursor.execute("""INSERT INTO FortalezaTemp VALUES (11353, 'Cuota de Prueba', 'Unidad', 2006, 1450, 10, 1100, 350, 125,  '001-001-0007592', '14-04-2023');""")
   connection.commit()
   print('Fila insertada')
except Exception as e:
   print(f"Error al insertar: {e}")

In [ ]:
cursor.close()
connection.close()

Extraer datos de la tabla de la factura PDF

In [ ]:
root_dir = os.getcwd()

print("Ruta base archivos: {}".format(root_dir)) 

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                # print(first_page.extract_table())

In [ ]:
diccColumnas = ('Codigo', 'Descripcion', 'Unidad_medida', 'Cantidad', 'Precio_Unitario', 'Descuento', 'Exentas', 'Monto_5', 'Monto_10', 'NroFactura', 'FechaEmision')
diccColumnas

In [ ]:
root_dir = os.getcwd()

df_archivos = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()
                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                    # Se agrega el Nro de Factura al Df
                texto = first_page.extract_text()
                nroFact  = texto[0:15]
                print("Nro Factura: {}".format(nroFact)) 
                df_t['NroFactura']=nroFact

                # #Se agrega la fecha y Hora impresa en la factura
                lines = texto.split('\n')
                filtered_lines = [line for line in lines if 'Fecha y hora' in line]
                # print('\n'.join(filtered_lines))
                str(filtered_lines)
                fechafact =str(filtered_lines)[27:37]
                print("Fecha Emision: {}".format(fechafact))
                df_t['FechaEmision']=fechafact
                df_archivos.append(df_t)
                
            df = pd.concat(df_archivos, ignore_index=True)
            df.columns = diccColumnas


In [ ]:
# Removemos los separadores de miles y el ultimo punto decimal para evitar montos incorrectos
columnasMontos = ['Precio_Unitario', 'Exentas', 'Monto_5', 'Monto_10']
df[columnasMontos] = df[columnasMontos].applymap(lambda x: x.replace('.', ''))
# recotamos 1 caracter a las columnas con valores en moneda
df[columnasMontos] = df[columnasMontos].applymap(lambda x: x[:-1])
df.head()

In [ ]:
# Se extrae los 3 caracteres para las columnas Cuantitativas
colCantidades = ['Cantidad', 'Descuento']
df[colCantidades] = df[colCantidades].applymap(lambda x: x[:-3])

In [ ]:
df_insert = df.tail()
df_insert

Metodo 1, insertando con Cursor de Pyodbc

In [ ]:
# Create a cursor from the connection
cursor = connection.cursor()

for index, row in df_insert.iterrows():
   cursor.execute("INSERT INTO FortalezaTemp (Codigo,Descripcion,Unidad_medida,Cantidad, Precio_Unitario,Descuento,Exentas, Monto_5, Monto_10, NroFactura, FechaEmision) values(?,?,?,?,?,?,?,?,?,?,?)", row.Codigo, row.Descripcion, row.Unidad_medida, row.Cantidad, row.Precio_Unitario, row.Descuento, row.Exentas,row.Monto_5, row.Monto_10, row.NroFactura, row.FechaEmision)
connection.commit()

Metodo 2 Insert con libreria Pandas

In [ ]:
query = "SELECT * FROM dbo.FortalezaTemp"
df_query = pd.read_sql(query, connection)
df_query.tail()

### Bloque para leer la factura electronica version 1
Comprende de Marzo de 2022 a Agosto 2022

In [ ]:
rutaEspec = r"C:\\Users\\rfigu\\Pictures\\Fortaleza\\facturaElectronicasV1"

df_archivos_v1 = []
ciclo = 0

for dirpath, dirnames, filenames in os.walk(rutaEspec):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_paths = os.path.join(dirpath, filename)
            # Se utiliza pdfPlumber para leer los datos de la tabla factura
            with pdfplumber.open(file_paths) as temp:
                print(file_paths)
                ciclo += 1
                    # Nombre de pdf dinamico
                print(f"pdf_{ciclo}")
                first_page = temp.pages[0]
                        # print(first_page.extract_table())
                dfnames = first_page.extract_table()
                df_t = pd.DataFrame(dfnames)
                headers = df_t.iloc[0].values
                df_t.columns = headers
                df_t.drop(index=0, axis=0, inplace=True)
                df_t = df_t.dropna()

                    # Se agrega el Nro de Factura al Df # factura version 1
                texto = first_page.extract_text()
                nroFact  = texto[327:342]
                print("Nro Factura: {}".format(nroFact)) 
                df_t['NroFactura']=nroFact

                #   #Se agrega la fecha y Hora impresa en la factura version 1
                lines = texto.split('\n')
                filtered_lines = [line for line in lines if 'Fecha de Emisión:' in line]
                fechafact =str(filtered_lines)[20:30]
                print("Fecha Emision: {}".format(fechafact))
                df_t['FechaEmision']=fechafact
                df_archivos_v1.append(df_t)
                            # if ciclo == 2:
                            #     break    # break here
                
            df_v1 = pd.concat(df_archivos_v1, ignore_index=True)
            df_v1.columns = diccColumnas

In [ ]:
lines = texto.split('\n')
filtered_lines = [line for line in lines if 'Fecha de Emisión:' in line]
fechafact =str(filtered_lines)[20:30]
print("Fecha Emision: {}".format(fechafact))